<a href="https://colab.research.google.com/github/lalva224/Butterfly-Detection/blob/main/Butterfly_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install roboflow jsonlines groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 2.6 MB/s eta 0:00:00


In [2]:
import os
import shutil
import pandas as pd
from roboflow import Roboflow
from google.colab import userdata
from google.colab import drive



In [3]:
# connect your roboflow api
rf = Roboflow(userdata.get('ROBOFLOW_API_KEY'))


In [4]:
# connect drive storage inside the google colab session
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#ONLY FOR UPLOADING NEW PROJECTS
# paste roboflow download code here



loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...


In [ ]:
#ONLY FOR UPLOADING NEW PROJECTS
#this will get directories inside current session. This will be used to then move inside the drive storage permanently below
def get_butterfly_directories_for_drive_upload():
  directories = []
  exclude = {'sample_data','drive','.config'}
  for folder in os.listdir('/content/'):
    if folder not in exclude:
      directories.append(f'/content/{folder}')
  return directories

butterfly_directories = get_butterfly_directories_for_drive_upload()





In [ ]:
butterfly_directories

['/content/kupu2-dan-ngengat-klasifikasi-2',
 '/content/Butterfly-12',
 '/content/butterfly-1']

In [ ]:
##ONLY FOR UPLOADING NEW PROJECTS
#use only for uploading folder in cwd to drive
import os

# training dataset folder
SHARED_FOLDER_PATH = "/content/drive/MyDrive/Butterfly-Datasets"

# move datasets to google drive
def move_to_drive(dataset_path, destination_folder):
    destination_path = os.path.join(destination_folder, os.path.basename(dataset_path))
    if os.path.exists(destination_path):
        print(f"Dataset {os.path.basename(dataset_path)} already exists in the destination folder. Skipping move.")
    else:
        shutil.move(dataset_path, destination_folder)
        print(f"Moved dataset from {dataset_path} to {destination_folder}")

for dataset in butterfly_directories:
    move_to_drive(dataset, SHARED_FOLDER_PATH)


Dataset kupu2-dan-ngengat-klasifikasi-2 already exists in the destination folder. Skipping move.
Dataset Butterfly-12 already exists in the destination folder. Skipping move.
Dataset butterfly-1 already exists in the destination folder. Skipping move.


In [5]:
# when not uploading to drive, use this function to get the butterfly directories inside drive, to be used for getting the butterfly names

def get_butterfly_directories_inside_drive():
  directories = []
  exclude = {'sample_data','drive','.config'}
  for folder in os.listdir('/content/drive/MyDrive/Butterfly-Datasets'):
    if folder not in exclude:
      directories.append(f'/content/drive/MyDrive/Butterfly-Datasets/{folder}')
  return directories

butterfly_directories = get_butterfly_directories_inside_drive()
butterfly_directories

['/content/drive/MyDrive/Butterfly-Datasets/classify_butterfly_and_moth-3',
 '/content/drive/MyDrive/Butterfly-Datasets/butterfly-1',
 '/content/drive/MyDrive/Butterfly-Datasets/Butterfly--2',
 '/content/drive/MyDrive/Butterfly-Datasets/butterflies-2',
 '/content/drive/MyDrive/Butterfly-Datasets/detecton-Butterfly-1',
 '/content/drive/MyDrive/Butterfly-Datasets/CV-2',
 '/content/drive/MyDrive/Butterfly-Datasets/PDI-PROYECTO-1',
 '/content/drive/MyDrive/Butterfly-Datasets/butterfly-class-1',
 '/content/drive/MyDrive/Butterfly-Datasets/yolov-Butterfly-2',
 '/content/drive/MyDrive/Butterfly-Datasets/Butterfly-12',
 '/content/drive/MyDrive/Butterfly-Datasets/kupu2-dan-ngengat-klasifikasi-2',
 '/content/drive/MyDrive/Butterfly-Datasets/Butterfly-dt2-2',
 '/content/drive/MyDrive/Butterfly-Datasets/.ipynb_checkpoints']

In [9]:
# combine datasets
import jsonlines
def get_butterflies():
  butterflies = []
  exclude = {'.ipynb_checkpoints'}
  for folder in butterfly_directories:
    if folder not in exclude:
      if os.path.isdir(f'{folder}/train'):
        directories = []
        for directory in os.listdir(f'{folder}/train'):
          if '.jpg' in directory:
            directories.append(directory.split('--')[0])
          else:
            directories.append(directory)
        butterflies.extend(directories)
    else:
      with jsonlines.open(f'{folder}/_annotations.train.jsonl','r') as file:
        data =  [obj['messages'][3]['content'].split(' ')[-1] for obj in file]
      butterflies.extend(data)
  return butterflies


butterflies = set(get_butterflies())

butterflies

{'88 AN',
 '88 BANDED GOLD',
 'ADMIRAL RED',
 'ADONIS',
 'AFRICAN GIANT SWALLOWTAIL',
 'AMERICAN SNOOT',
 'AN 88',
 'APPOLLO',
 'ARCIGERA FLOWER MOTH',
 'ARGUS BROWN',
 'ATALA',
 'ATLAS MOTH',
 'AWL BANDED COMMON',
 'American-Copper',
 'BANDED GOLD',
 'BANDED HELICONIAN ORANGE',
 'BANDED MOTH TIGER',
 'BANDED ORANGE HELICONIAN',
 'BANDED PEACOCK',
 'BANDED TIGER MOTH',
 'BARRED FLASHER TWO',
 'BECKERS WHITE',
 'BIRD CHERRY ERMINE MOTH',
 'BIRD HAWK HUMMING MOTH',
 'BIRDWING BROOKES',
 'BIRDWING CAIRNS',
 'BLACK HAIRSTREAK',
 'BLUE CHALK HILL',
 'BLUE CROW SPOTTED',
 'BLUE MORPHO',
 'BLUE SPOTTED CROW',
 'BROOKES BIRDWING',
 'BROWN ARGUS',
 'BROWN SIPROETA',
 'BURNET MOTH SIXSPOT',
 'Blue Mormon',
 'CABBAGE WHITE',
 'CAIRNS BIRDWING',
 'CATTLEHEART CELLED GREEN',
 'CHALK HILL BLUE',
 'CHECQUERED SKIPPER',
 'CHESTNUT',
 'CINNABAR MOTH',
 'CLEARWING MOTH',
 'CLEOPATRA',
 'CLOAK MOURNING',
 'CLODIUS PARNASSIAN',
 'CLOUDED SULPHUR',
 'COMA EASTERN',
 'COMET MOTH',
 'COMMON BANDED AWL',
 'CO

In [21]:
#use LLM to improve the make the labeling the same
from groq import Groq
import json
client = Groq(api_key=userdata.get('GROQ_API_KEY'))

system_prompt = f"""
Here is a list of butterflies: {butterflies}
They are not normalized under the same format, for instance some are capitlized like : ZEBRA LONG WING and some have hyphens like Zebra-Longwing. What is required is a lower case non hyphen format
Like this:  tailed jay

Your response must be json under this format:
{{
  'butterflies': []
}}
"""
completion = client.chat.completions.create(
    model = "llama-3.3-70b-versatile",
    messages = [{"role":"system","content":system_prompt}],
    response_format = {"type":"json_object"}
)


In [30]:
butterflies = set(json.loads(completion.choices[0].message.content)['butterflies'])
butterflies

{'admiral red',
 'adonis',
 'african giant swallowtail',
 'american copper',
 'american snoot',
 'appollo',
 'arcigera flower moth',
 'argus brown',
 'atala',
 'atlas moth',
 'awl banded common',
 'banded gold',
 'banded heliconian orange',
 'banded moth tiger',
 'banded orange heliconian',
 'banded peacock',
 'banded tiger moth',
 'barred flasher two',
 'beckers white',
 'bird cherry ermine moth',
 'bird hawk humming moth',
 'birdwing brookes',
 'birdwing cairns',
 'black hairstreak',
 'blue chalk hill',
 'blue crow spotted',
 'blue mormon',
 'blue morpho',
 'blue spotted crow',
 'brookes birdwing',
 'brown argus',
 'brown siproeta',
 'burnet moth sixspot',
 'cabbage white',
 'cairns birdwing',
 'cattleheart celled green',
 'chalk hill blue',
 'chequered skipper',
 'chestnut',
 'cinnabar moth',
 'cladius parnassian',
 'clearwing moth',
 'cleopatra',
 'clouded sulphur',
 'coma eastern',
 'comet moth',
 'common banded awl',
 'common banded peacock',
 'common buckeye',
 'common mime',
 '

In [31]:
south_fl_butterflies = [
    "Aaron’s Skipper",
    "American (Painted) Lady",
    "Amethyst Hairstreak",
    "Atala",
    "Bahamian Swallowtail",
    "Banded Hairstreak",
    "Barred Yellow (Barred Sulphur)",
    "Bartram’s Scrub-Hairstreak",
    "Black Swallowtail",
    "Brazilian Skipper",
    "Carolina Satyr",
    "Cassius Blue",
    "Ceraunus Blue",
    "Checkered White",
    "Clouded Skipper",
    "Cloudless Sulphur",
    "Common Buckeye",
    "Common/White Checkered-Skipper",
    "Cuban Crescent",
    "Dainty Sulphur",
    "Delaware Skipper",
    "Dingy Purplewing",
    "Dina Yellow",
    "Dorantes Longtail",
    "Eastern Black Swallowtail",
    "Eastern Pygmy-Blue",
    "Eastern Tiger Swallowtail",
    "Eufala Skipper",
    "Fiery Skipper",
    "Florida Duskywing",
    "Florida Leafwing",
    "Florida Purplewing",
    "Florida White",
    "Fulvous Hairstreak",
    "Georgia Satyr",
    "Giant Swallowtail",
    "Gray Hairstreak",
    "Gray Ministreak",
    "Great Purple Hairstreak",
    "Great Southern White",
    "Gulf Fritillary",
    "Hackberry Emperor",
    "Hammock Skipper",
    "Horace's Duskywing",
    "Julia",
    "Julia Heliconian",
    "Large Orange Sulphur",
    "Least Skipper",
    "Little Metalmark",
    "Little Yellow (Little Sulphur)",
    "Long-tailed Skipper",
    "Lyside Sulphur",
    "Malachite",
    "Mangrove Buckeye",
    "Mangrove Skipper",
    "Martial Scrub-Hairstreak",
    "Mallow Scrub-Hairstreak",
    "Mimosa Yellow",
    "Miami Blue",
    "Monarch",
    "Nickerbean Blue",
    "Ocola Skipper",
    "Obscure Skipper",
    "Orange-barred Sulphur",
    "Palamedes Swallowtail",
    "Palatka Skipper",
    "Painted Lady",
    "Pearl Crescent",
    "Phaon Crescent",
    "Pink-spot Sulphur",
    "Pipevine Swallowtail",
    "Polydamas Swallowtail",
    "Queen",
    "Question Mark",
    "Ruddy Daggerwing",
    "Red Admiral",
    "Red-banded Hairstreak",
    "Red-spotted Purple",
    "Sachem",
    "Salt Marsh Skipper",
    "Schaus’ Swallowtail",
    "Silver-banded Hairstreak",
    "Silver-spotted Skipper",
    "Sleepy Orange",
    "Soldier",
    "Southern Broken-Dash",
    "Southern Dogface",
    "Southern Skipperling",
    "Spicebush Swallowtail",
    "Statira Sulphur",
    "Three-spotted Skipper",
    "Tiger Swallowtail, Eastern",
    "Tropical Buckeye",
    "Tropical Checkered-Skipper",
    "Twin-spot Skipper",
    "Variegated Fritillary",
    "Viceroy",
    "White M Hairstreak",
    "White Peacock",
    "Whirlabout",
    "Zarucco Duskywing",
    "Zebra (Longwing) Heliconian",
    "Zebra Heliconian",
    "Zebra Swallowtail"
]





south_fl_butterflies = [butterfly.lower() for butterfly in south_fl_butterflies]
butterflies = [butterfly.lower() for butterfly in butterflies]

In [32]:
# count the percentage of butterflies from the South Florida list
total_length = len(south_fl_butterflies)
count = 0
for butterfly in butterflies:
  if butterfly in south_fl_butterflies:
    count+=1

total = f'{count}/{total_length}'
total



'13/104'